<a href="https://colab.research.google.com/github/Tong00020/PLN---2022/blob/main/Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Atividade 3: Recuperação de Textos

Feito por:
Lucas De Almeida Bandeira Macedo 190047089
Tong Zhou 190038764

In [20]:
import nltk
import pandas as pd
import re
import numpy as np
import math
from nltk.stem.porter import *

nltk.download('reuters')
from nltk.corpus import reuters

cats = reuters.categories()
print("Reuters has %d categories:\n%s" % (len(cats), cats))

fileids = reuters.fileids()

Reuters has 90 categories:
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [21]:
categories = []
text = []

for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

df = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})
df

,ids,categories,text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...
...,...,...,...
10783,training/999,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
10784,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...
10785,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...
10786,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...


In [22]:
texts = df['text'][0:5000]
texts

0       ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1       CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2       JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3       THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n  ...
4       INDONESIA SEES CPO PRICE RISING SHARPLY\n  Ind...
                              ...                        
4995    FREEDOM SAVINGS &lt;FRDM> MAY SEEK SALE OR MER...
4996                       26-FEB-1987\n  26-FEB-1987\n\n
4997    HARPER INTERNATIONAL INC &lt;HNT> 3RD QTR JAN ...
4998    BRAZIL MAKES SECOND AMAZON OIL DISCOVERY\n  Th...
4999    IBC CLOSES EXPORT REGISTRATIONS - EXPORTERS\n ...
Name: text, Length: 5000, dtype: object

In [23]:
def processamento_de_texto(t : str):
  """Função para fazer o pré-processamento de textos. A função segue a seguinte ordem:
  Transforma string em lower case
  Separa a string em tokens, removendo caracteres especiais
  Substitui algumas palavras e acrônimos pelos seus significados


  Args:
      texto (str): _description_

  Returns:
      List[str]: Retorna lista de tweets com palavras processadas
  """
  t = t.lower()
  t = re.sub(r'[\#\,\+\=\-\_\?\[\]\{\}\@\.\'\$\!\&\>\<\;\"\(\)\:\/\\\*\^]+',' ',t)
  t = re.sub(r'[0-9]+',' ',t)
  
  tokens = re.split(r'[ ]+', t)
  stopword_list_nltk = nltk.corpus.stopwords.words("english")
  clean_words = []
  for token in tokens: # remocao de stop words
    if token not in stopword_list_nltk:
      clean_words.append(token)
  stemmer = PorterStemmer()
  stem = [stemmer.stem(t) for t in clean_words]
  return " ".join(stem)

In [24]:
corpus = texts.apply(processamento_de_texto)

corpus

0       asian export fear damag u japan rift\n mount t...
1       china daili say vermin eat pct grain stocks\n ...
2       japan revis long term energi demand downwards\...
3       thai trade deficit widen first quarter\n thail...
4       indonesia see cpo price rise sharply\n indones...
                              ...                        
4995    freedom save lt frdm may seek sale merger\n fr...
4996                                      feb \n feb \n\n
4997    harper intern inc lt hnt rd qtr jan \n shr ct ...
4998    brazil make second amazon oil discovery\n braz...
4999    ibc close export registr exporters\n brazilian...
Name: text, Length: 5000, dtype: object

In [25]:
bow_ponderado = {}
N = {} # quantidade de termos em cada documento

for i,doc in enumerate(corpus):
    bow_ponderado['texto {}'.format(i+1)] = dict()
    N['texto {}'.format(i+1)] = len(doc.split())
    for word in doc.lower().split():
        if word in bow_ponderado['texto {}'.format(i+1)]:
            bow_ponderado['texto {}'.format(i+1)][word] += 1
        else:
            bow_ponderado['texto {}'.format(i+1)][word] = 1

for doc in bow_ponderado:
    for word in bow_ponderado[doc]:
        bow_ponderado[doc][word] /= N[doc]

# bow_ponderado

In [26]:
frase = " ".join(corpus)
tokens = frase.split()
vocab = sorted(set(tokens))
pd.Series(vocab).to_csv("vocabulario.csv")

In [27]:
idf = {}
for i, sent in enumerate(corpus):
  for wrd in set(sent.lower().split()):
    if wrd not in idf:
      idf[wrd] = 1
    else:
      idf[wrd] += 1
# for word in idf:
idf = pd.Series(idf)
idf = np.log(len(corpus)/idf)
idf

open        2.991740
avow        8.517193
mln         0.781760
possibl     3.170086
exports     5.521461
              ...   
sturm       8.517193
firearm     8.517193
wesson      8.517193
uruc        8.517193
kilometr    8.517193
Length: 17624, dtype: float64

In [28]:
df_term_frequency = pd.DataFrame().from_records(bow_ponderado).fillna(0).T
df_term_frequency

,asian,export,fear,damag,u,japan,rift,mount,trade,friction,...,fhlb,bfi,ceco,leung,willard,whatsoev,hm,chronar,crnr,capitalized
texto 1,0.004158,0.018711,0.004158,0.004158,0.039501,0.027027,0.002079,0.002079,0.033264,0.002079,...,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
texto 10,0.000000,0.000000,0.000000,0.000000,0.009732,0.004866,0.000000,0.000000,0.000000,0.000000,...,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
texto 100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
texto 1000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
texto 1001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
texto 995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
texto 996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
texto 997,0.000000,0.000000,0.000000,0.000000,0.008197,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.02459,0.008197,0.016393,0.008197,0.008197,0.00000,0.000000,0.000000,0.000000
texto 998,0.000000,0.000000,0.000000,0.000000,0.005650,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00565,0.000000,0.000000,0.000000


In [29]:
df_term_frequency = df_term_frequency.multiply(idf)
df_term_frequency

,a,aa,aaa,aabex,aac,aachen,aaica,aaix,aam,aar,...,zones,zoran,zro,zuccherifici,zuheir,zurich,zuyuan,zwermann,zy,üside
texto 1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 10,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 100,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 1000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 1001,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
texto 995,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 997,0.014563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 998,0.020075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
def cosine_similarity(vetor1,vetor2):
    
    prod_interno = (vetor1*vetor2).sum()

    norma_vetor1 = math.sqrt((vetor1*vetor1).sum())
    norma_vetor2 = math.sqrt((vetor2*vetor2).sum())

    return prod_interno/(norma_vetor1*norma_vetor2)

In [36]:
n_doc = df_term_frequency.shape[0]
# n_doc = 2
ranking = [0]*10
for j in range(n_doc):
    res = []
    for i in range(df_term_frequency.shape[0]):
        res.append(cosine_similarity(df_term_frequency.loc[f"texto {j+1}"], df_term_frequency.loc[f"texto {i+1}"]))

    top_10000 = pd.Series(res).sort_values(ascending=False)

    top_10 = top_10000.head(11)
    cat_orig = df.iloc[top_10.index[0]].categories[0]

    for i in range(1, 11):
        ranking[i-1] += cat_orig in df.iloc[top_10.index[i]].categories

ranking

[4034, 3792, 3715, 3717, 3665, 3577, 3559, 3538, 3515, 3433]

In [37]:
acc = 0
for number in ranking:
  acc += number/n_doc

acc /= 10
acc

0.7309000000000001

In [38]:
pd.Series(ranking).to_csv("output.csv")